> [back](../README.md)

# gs-rs [ˈdʒiːzrs] 

## Contact
Samuel Valenzuela, Florian Rohm , Dr. Daniel Pape, daniel.pape(at)tngtech.com

TNG Technology Consulting GmbH


## Abstract
**gs-rs** is a Rust library for the optimization of non-linear least squares problems embeddable
as a (hyper)graph.

It is suitable for the optimization of an error function with respect to a set of parameters as in
SLAM (simultaneous localization and mapping) or BA (bundle adjustment).


## Background Literature
The optimization algorithm used by **gs-rs** is based on parts of **g2o**. This project should suffice to understand the optimization's implementation in **gs-rs**. 
The following papers by the developers of **g2o** are recommended if a deeper 
understanding of the theory behind the algorithm is of interest:
* _g2o: A General Framework for Graph Optimization_, Kümmerle et al.: This paper documents the derivation of the algorithm's structure.
* _A Tutorial on Graph-Based SLAM_, Grisetti et al.: This paper contains additional comments on the calculations in 2D and 3D. Here it is presented how the least squares optimization works on a manifold.


## Hypergraph-Embeddable Optimization Problems
### Least Squares Optimization Problems
#### Definition
A least squares optimization problem with a set of constraints $\mathcal{C}$ can be generally formulated as:

\begin{align}
    F(\chi) &= \sum_{k\in\mathcal{C}}e_k(\chi_k, z_k)^{T}\Omega_k e_k(\chi_k, z_k)\\
    {\chi}^* &= \text{argmin}_{\chi} F(\chi),
\end{align}

where

\begin{align}
    \chi = (\chi_1^T,\dots,\chi_n^T)^T
\end{align}

is the structured vector of parameters subject to optimization and

\begin{align}
    \chi_k = (\chi_{k_1}^T,\dots,\chi_{k_q}^T)^T
\end{align}

is the structured vector of parameter blocks subject to the constraint $k\in\mathcal{C}$.

$z_k$ and $\Omega_k$ denote the mean and information matrix of constraint $k$, respectively.

Finally, $e_k(\chi_k, z_k)$ is a vector error function measuring how well the parameter blocks
$\chi_k$ satisfy the constraint $k$.

#### Linearization of the Error Function
Given an initial guess $\hat{\chi}$ of the parameter set $\chi$, we can expand the error function
by means of a first order _Taylor_ expansion:

\begin{align}
    e_k(\chi_k) &= e_k(\hat{\chi_k}+\Delta\chi_k)\\
    &\approx e_k(\hat{\chi_k})+J_{k}\Delta\chi_k,
\end{align}

where $J_{k}=\nabla e_k(\hat{\chi_k})$ is the 
_Jacobian_ computed at $\hat{\chi}_k$.

Insertion into formulation of the optimization problem yields:

\begin{align}
    F_k(\hat{\chi_k}+\Delta\chi_k) &= e_k(\hat{\chi_k}+\Delta\chi_k)^{T}\Omega_k e_k(\hat{\chi_k}+\Delta\chi_k)\\
    &\approx \left(e_k(\hat{\chi_k})+J_{k}\Delta\chi_{k}\right)^T\Omega_k\left(e_k(\hat{\chi_k})+J_{k}\Delta\chi_{k}\right)\\
    &= \underbrace{e_k(\hat{\chi_k})^T\Omega_k e_k(\hat{\chi_k})}_{c_k}
    + 2\underbrace{e_k(\hat{\chi_k})^{T}\Omega_{k}J_k}_{b_k}\Delta \chi_k
    + \Delta\chi_k^T \underbrace{J_k^T\Omega_k J_k}_{H_k}\Delta \chi_k\\
    &= c_k + 2b_{k}\Delta \chi_k + \Delta \chi_k^T H_k \Delta \chi_k.
\end{align}

Generalizing this result to the set of all constraints yields:

\begin{align}
    F(\hat{\chi}+\Delta \chi) &= \sum_{k\in\mathcal{C}} F_k(\hat{\chi_k}+ \Delta\chi_k)\\
    &= \sum_{k\in\mathcal{C}} c_k + 2b_{k}\Delta \chi_k + \Delta \chi_k^T H_k \Delta \chi_k\\
    &= c + 2b^T\Delta\chi + \Delta \chi^T H \Delta \chi.
\end{align}

where

\begin{align}
    c &= \sum_{k\in\mathcal{C}}c_{k}\\
    b &= \bigoplus_{k\in\mathcal{C}}b_{k}\\
    \Delta \chi &= \bigoplus_{k\in\mathcal{C}}\Delta \chi_{k}\\
    H &= \bigoplus_{k\in\mathcal{C}} H_{k}.
\end{align}

Here, $\bigoplus$ denotes an addition of blocks for vectors and matrices, where the position is derived from the constraint index.

We need to minimize $F(\hat\chi+\Delta\chi)$ in $\Delta\chi$. It can be done by solving the linear system

\begin{align}
    H\Delta\chi^\star = -b
\end{align}

#### Example
Let us consider a system of two parameter blocks $\chi_0$, $\chi_1$ (e.g. poses in two dimensions)
and three constraints $z_0$, $z_1$ (e.g. position measurements), $z_{01}$ (e.g. an odometry measurement), where the first two constraints
control only one parameter block each and the
last constraint connects the two parameter blocks.

We will set $z_{0}=(0,1)^T, z_1 = (1,0)^T, z_{01} = (0.5, -0.5)^T$.
Thus, we have a system, where the position measurements are further apart
than the odometry requires.

The information matrices are defined as follows:

\begin{align}
    \Omega_{0} &= \begin{pmatrix}
                      10 & 0 \\ 0 & 10
    \end{pmatrix}\\
    \Omega_{1} &= \begin{pmatrix}
                      10 & 0 \\ 0 & 10
    \end{pmatrix}\\
    \Omega_{01} &=\begin{pmatrix}
                      1 & 0 \\ 0 & 1
    \end{pmatrix}
\end{align}

That is, the odometry measurement is considered ten times less precise than the position measurement.

We will define the error functions

\begin{align}
    e_0(\chi_0,z_0) &= z_0 - \chi_0 \\
    e_1(\chi_1,z_1) &= z_1 - \chi_1 \\
    e_{01}(\chi_{01},z_{01}) &= z_{01} - (\chi_1 - \chi_0).
\end{align}

Thus, for the _Jacobian_'s we find

\begin{align}
J_0 &= \begin{pmatrix}
           \frac{\partial (z_0 - \chi_0)_x}{\partial (\chi_0)_x} & \frac{\partial (z_0 - \chi_0)_x}{\partial (\chi_0)_y} \\
           \frac{\partial (z_0 - \chi_0)_y}{\partial (\chi_0)_x} & \frac{\partial (z_0 - \chi_0)_y}{\partial (\chi_0)_y}
\end{pmatrix}\newline
&= \begin{pmatrix}
       1 & 0 \\ 0 & 1
\end{pmatrix}\newline
J_1 &= \begin{pmatrix}
           1 & 0 \\ 0 & 1
\end{pmatrix}\newline
J_{01} &= \begin{pmatrix}
              \frac{\partial (z_0 - (\chi_1 - \chi_0))_x}{\partial (\chi_0)_x} &
              \frac{\partial (z_0 - (\chi_1 - \chi_0))_x}{\partial (\chi_0)_y} &
              \frac{\partial (z_0 - (\chi_1 - \chi_0))_x}{\partial (\chi_1)_x} &
              \frac{\partial (z_0 - (\chi_1 - \chi_0))_x}{\partial (\chi_1)_y} \\
              \frac{\partial (z_0 - (\chi_1 - \chi_0))_y}{\partial (\chi_0)_x} &
              \frac{\partial (z_0 - (\chi_1 - \chi_0))_y}{\partial (\chi_0)_y} &
              \frac{\partial (z_0 - (\chi_1 - \chi_0))_y}{\partial (\chi_1)_x} &
              \frac{\partial (z_0 - (\chi_1 - \chi_0))_y}{\partial (\chi_1)_y}
\end{pmatrix}\\
&= \begin{pmatrix}
       -1 & 0 & 1 & 0 \\ 0 & -1 & 0 & 1
\end{pmatrix}
\end{align}

Let us apply the position constraints as initial guesses of the pose variables:

\begin{align}
    \hat{\chi_0} &= z_0 = (0,1)^{T}\\
    \hat{\chi_1} &= z_1 = (1,0)^{T}.
\end{align}

Now we can calculate the components of our target equation ?:

\begin{align}
    b_0 &= e_0^T \cdotp \Omega_0 J_{0}\\
    &= (0,0) \cdotp \begin{pmatrix}
                        10 & 0 \\ 0 & 10
    \end{pmatrix} \cdotp \begin{pmatrix}
                             1 & 0 \\ 0 & 1
    \end{pmatrix}\newline
    &= (0,0).\newline
    b_1 &= e_1^T \cdotp \Omega_{1} J_{1}\newline
    &= (0,0) \cdotp \begin{pmatrix}
                        10 & 0 \\ 0 & 10
    \end{pmatrix} \cdotp \begin{pmatrix}
                             1 & 0 \\ 0 & 1
    \end{pmatrix}\newline
    &= (0,0).\\
    b_{01} &= e_{01}^T \cdotp \Omega_{01} J_{01}\newline
    &= \left( (0.5, -0.5) - \left( (1,0) - (0,1) \right) \right)\cdotp \begin{pmatrix}
                                                                           1 & 0 \\ 0 & 1
    \end{pmatrix} \cdotp \begin{pmatrix}
                             1 & 0 & -1 & 0 \\ 0 & 1 & 0 & -1
    \end{pmatrix}\newline
    &= (-0.5, 0.5)\cdotp \begin{pmatrix}
                             1 & 0 \\ 0 & 1
    \end{pmatrix} \cdotp \begin{pmatrix}
                             1 & 0 & -1 & 0 \\ 0 & 1 & 0 & -1
    \end{pmatrix}\newline
    &= (-0.5, 0.5, 0.5, -0.5).\newline
    \Rightarrow b &= b_0 \oplus b_1 \oplus b_{01}\newline
    &= (0,0) \oplus (0,0) \oplus (-0.5, 0.5, 0.5, -0.5)\newline
    &= (-0.5, 0.5, 0.5, -0.5)\newline
    H_0 &= J_{0}^T \Omega_0 J_{0}\newline
    &= \begin{pmatrix}
           -1 & 0 \\ 0 & -1
    \end{pmatrix} \cdotp \begin{pmatrix}
                             10 & 0 \\ 0 & 10
    \end{pmatrix} \cdotp \begin{pmatrix}
                             -1 & 0 \\ 0 & -1
    \end{pmatrix}\newline
    &= \begin{pmatrix}
           10 & 0 \\ 0 & 10
    \end{pmatrix}.\newline
    H_1 &= J_{1}^T \Omega_1 J_{1}\newline
    &= \begin{pmatrix}
           10 & 0 \\ 0 & 10
    \end{pmatrix}.\newline
    H_{01} &= J_{01}^T \Omega_{01} J_{01}\newline
    &= \begin{pmatrix}
           1 & 0 \\ 0 & 1 \\ -1 & 0 \\ 0 & -1
    \end{pmatrix} \cdotp \begin{pmatrix}
                             1 & 0 \\ 0 & 1
    \end{pmatrix} \cdotp \begin{pmatrix}
                             1 & 0 & -1 & 0 \\ 0 & 1 & 0 & -1
    \end{pmatrix}\newline
    &= \begin{pmatrix}
           1 & 0 & -1 & 0\\ 0 & 1 & 0 & -1 \\ -1 & 0 & 1 & 0 \\ 0 & -1 & 0 & 1
    \end{pmatrix}.\newline
    \Rightarrow H &= H_0 \oplus H_1 \oplus H_{01}\\
    &= \begin{pmatrix}
           10 & 0 \\ 0 & 10
    \end{pmatrix} \oplus \begin{pmatrix}
                             10 & 0 \\ 0 & 10
    \end{pmatrix} \oplus \begin{pmatrix}
                             1 & 0 & -1 & 0\\ 0 & 1 & 0 & -1 \\ -1 & 0 & 1 & 0 \\ 0 & -1 & 0 & 1
    \end{pmatrix}\newline
    & = \begin{pmatrix}
            11 & 0 & -1 & 0\\ 0 & 11 & 0 & -1 \\ -1 & 0 & 11 & 0 \\ 0 & -1 & 0 & 11
    \end{pmatrix}
\end{align}

Now, the last step is to solve

\begin{align}
    H\Delta \chi^* &= -b\newline
    \begin{pmatrix}
        11 & 0 & -1 & 0\\ 0 & 11 & 0 & -1 \\ -1 & 0 & 11 & 0 \\ 0 & -1 & 0 & 11
    \end{pmatrix} \Delta \chi^* &=
    \begin{pmatrix}
        0.5 \\ -0.5 \\ -0.5 \\ 0.5
    \end{pmatrix} \newline
    \Rightarrow \Delta \chi^* &= \frac{1}{24} \begin{pmatrix}
                                                  1 \\ -1 \\ -1 \\ 1
    \end{pmatrix}.
\end{align}

and apply the update

\begin{align}
    \chi^{(1)} &= \hat{\chi} + \Delta \chi^{*}\\
    &= (0,1,1,0)^T + \frac{1}{24}(1, -1, -1, 1)^{T}\\
    &\approx (0.04, 0.96, 0.96, 0.04).
\end{align}

Thus, our pose variables moved a little closer together, driven by the odometry constraint.



## Optimization Algorithm
In the following sections, the optimization algorithm will be introduced. Firstly, the structure's main
iteration steps will be introduced. Most of the work happens in the first step, the calculation of
$H$ and $b$. Subsequently, the calculations which apply to all dimensions and factor types will be presented.

### Iteration Steps
Given a specific number of iterations $n$ and the initial guess $x_i^{(0)}$ for each variable, the optimizer algorithm will repeat the following steps $n$ times:

1. Calculate $H$ and $b$ by setting them to $\boldsymbol{0}$, then looping through all factors and updating their non-fixed variables' entries in $H$ and $b$.
2. Calculate $\Delta x$, the vector containing data about how much each current variable guess $x_i^{(k)}$ should be updated in this step, by solving the linear system
\begin{align}
	H \Delta x = -b^T.
\end{align}
3. Update the guesses for each non-fixed variable $x_i$ with
\begin{align}
	x_i^{(k+1)} = x_i^{(k)} + \Delta x_i.
\end{align}

In the case of 2D variables with a rotation, normalize the rotation to $[-\pi, \pi)$.
In the case of 3D variables with a rotation, normalize the rotation of $\Delta x_i$. 
As explained in a later section, the quaternion data will only include the imaginary part. 
Therefore, set the scalar part to $1$ before normalizing. The $+$ operator then resembles the concatenation of two isometries, 
analogously as the $\circ$ operator later.



### Calculation of $\boldsymbol{H}$ and $\boldsymbol{b}$
How exactly the variables' entries in $H$ and $b$ are updated depends on the factor type. In all cases the factor's increments on 
parts of $H$ and $b$, $H^{\text{fac}}$ and $b^{\text{fac}}$,
respectively, will be computed as follows:

\begin{align}
H^{\text{fac}} = J^T \cdot \Omega \cdot J,
\end{align}

\begin{align}
b^{\text{fac}} = e^T \cdot \Omega \cdot J,
\end{align}

where $\Omega$, $J$ and $e$ are the factor's information matrix, Jacobian matrix and error vector, respectively. While $\Omega$ 
is a given constant of the factor, $J$ and $e$ have to be 
calculated for each factor in each iteration.

If the factor only involves the variable $x_i$, $H$ and $b$ are updated as follows:

\begin{align}
H_{ii} = H_{ii} + H^{\text{fac}},
\end{align}

\begin{align}
b_i = b_i + b^{\text{fac}},
\end{align}

where the subscripts of $H$ and $b$ denote the row and column index of the submatrix or subvector assigned to the respective variable. 
If the factor involves two variables $x_i$ and
$x_j$, $H^{\text{fac}}$ and $b^{\text{fac}}$ will have the structure

\begin{align}
H^{fac} =
\begin{pmatrix}
\boldsymbol{H}^{\text{fac}}_{ii} & \boldsymbol{H}^{\text{fac}}_{ij}\\
\boldsymbol{H}^{\text{fac}}_{ji} & \boldsymbol{H}^{\text{fac}}_{jj}
\end{pmatrix}
\end{align}

and

\begin{align}
b^{fac} =
\begin{pmatrix}
\boldsymbol{b}^{fac}_{i} & \boldsymbol{b}^{fac}_{j}
\end{pmatrix},
\end{align}

respectively, such that $H_{mn}$ will be incremented by $H^{\text{fac}}_{mn}$ and $b_n$ 
will be incremented by $b^{\text{fac}}_n$.
Fixed variables are excluded from $H$ and $b$ and therefore do not have any submatrices or subvectors which would need to be updated. 
In this case, these parts of $H^{\text{fac}}$ and $b^{\text{fac}}$
are simply ignored.

In the following sections, the calculation is described for all 2D and 3D factors supported by **gs-rs**.

## Optimization in 2D
In the following sections, the individual 2D factors' calculations of $J$ and $e$ are presented. The functions $\text{pos}(x)$ and $\text{rot}(x)$ will be used to refer to the 2D position vector and the rotation angle of a 2D pose $x$, respectively. 
Similarly, the functions $\text{pos}_x(x)$ and $\text{pos}_y(x)$ will be used to refer to the single value within the respective dimension.

Furthermore, the rotation matrices are denoted such that

\begin{align}
	R_\alpha =
	\begin{pmatrix}
		\text{cos}(\alpha) & -\text{sin}(\alpha)\\
		\text{sin}(\alpha) &  \text{cos}(\alpha)
	\end{pmatrix}
\end{align}

equals the rotation matrix with the angle $\alpha$ in 2D, and

\begin{align}
	R_z(\alpha) =
	\begin{pmatrix}
		\text{cos}(\alpha) & -\text{sin}(\alpha) & 0\\
		\text{sin}(\alpha) &  \text{cos}(\alpha) & 0\\
		          0 &            0 & 1
	\end{pmatrix}
\end{align}

equals the rotation matrix for a rotation around the $z$ axis with the angle $\alpha$ in 3D.

### Position2D
The _Position2D_ factor involves one _VehicleVariable_ $x_v$. 
The Jacobian matrix $J$ in this case is

\begin{align}
	J = R_z(-\text{rot}(x_v)).
\end{align}

Given the measurement $x_m$, the error vector

\begin{align}
	e = R_{-\text{rot}(x_m)} \circ (\text{pos}(x_v) - \text{pos}(x_m))
\end{align}

can be computed as well.



### Odometry2D

The _Odometry2D_ factor involves two _VehicleVariable_s $x_i$ and $x_j$. 
Given the measurement $x_{ij}$, the Jacobian matrix $J$ is calculated as follows:

\begin{align}
	\Delta x_{ij} = x_j - x_i
\end{align}

\begin{align}
	\text{sin}_i = \text{sin}(\text{rot}(x_i))
\end{align}

\begin{align}
	\text{cos}_i = \text{cos}(\text{rot}(x_i))
\end{align}

\begin{align}
	J_i = R_z(-\text{rot}(x_{ij})) \cdot
	\begin{pmatrix}
		-\text{cos}_i & -\text{sin}_i & -\text{sin}_i\cdot\text{pos}_x(\Delta x_{ij}) + \text{cos}_i\cdot\text{pos}_y(\Delta x_{ij})\\
		 \text{sin}_i & -\text{cos}_i & -\text{cos}_i\cdot\text{pos}_x(\Delta x_{ij}) - \text{sin}_i\cdot\text{pos}_y(\Delta x_{ij})\\
		     0 &      0 &                                   -1
	\end{pmatrix}
\end{align}

\begin{align}
	J_j = R_z(-\text{rot}(x_{ij})) \cdot R_z(-\text{rot}(x_i))
\end{align}

\begin{align}
	J =
	\begin{pmatrix}
		\boldsymbol{J_i} & \boldsymbol{J_j}
	\end{pmatrix}
\end{align}

The error vector $e$ is computed as follows:

\begin{align}
	e_{pos} = R_{-\text{rot}(x_{ij})} \cdot (R_{-\text{rot}(x_i)} \cdot \text{pos}(\Delta x_{ij}) - \text{pos}(x_{ij}))
\end{align}

\begin{align}
	e_{rot} = \text{rot}(\Delta x_{ij}) - \text{rot}(x_{ij})
\end{align}

After normalizing $e_{rot}$ to $[-\pi, \pi)$ as $e_{rot}^0$ the full error vector can be constructed with

\begin{align}
	e =
	\begin{pmatrix}
		\boldsymbol{e_{\text{pos}}}\\
		e_{\text{rot}}^0
	\end{pmatrix}
	.
\end{align}



### Observation2D
The `Observation2D` factor involves one `VehicleVariable` $x_i$ and one `LandmarkVariable` $x_j$. The measurement is denoted as $x_{ij}$, 
analogously to the previous section. Although $x_j$ and $x_{ij}$ are only positions rather than poses and therefore do not contain a rotation angle, the functions $\text{pos}(x)$, $\text{pos}_x(x)$ and $\text{pos}_y(x)$ will be used nevertheless to make the calculation path more understandable. Given the measurement $x_{ij}$, the Jacobian matrix $J$ is calculated as follows:

\begin{align}
	\text{pos}(\Delta x_{ij}) = \text{pos}(x_j) - \text{pos}(x_i)
\end{align}

\begin{align}
	\text{sin}_i = \text{sin}(\text{rot}(x_i))
\end{align}

\begin{align}
	\text{cos}_i = \text{cos}(\text{rot}(x_i))
\end{align}

\begin{align}
	J_i =
	\begin{pmatrix}
		-\text{cos}_i & -\text{sin}_i & -\text{sin}_i \cdot \text{pos}_x(\Delta x_{ij}) + \text{cos}_i \cdot \text{pos}_y(\Delta x_{ij})\\
		 \text{sin}_i & -\text{cos}_i & -\text{cos}_i \cdot \text{pos}_x(\Delta x_{ij}) - \text{sin}_i \cdot \text{pos}_y(\Delta x_{ij})
	\end{pmatrix}
\end{align}

\begin{align}
	J_j = R_{-\text{rot}(x_i)}
\end{align}

\begin{align}
	J =
	\begin{pmatrix}
		\boldsymbol{J_i} & \boldsymbol{J_j}
	\end{pmatrix}
\end{align}

The error vector $e$ is computed as follows:

\begin{align}
	e = R_{-\text{rot}(x_i)} \cdot \text{pos}(\Delta x_{ij}) - \text{pos}(x_{ij})
\end{align}

## Optimization in 3D

In 3D, rotations are often represented as quaternions.
To reduce these to a minimal representation, only the imaginary part of the unit quaternions is saved within the solution $\Delta x$,

In the following sections, the individual 3D factors' calculations of $J$ and $e$ are presented.
The functions $\text{pos}(x)$ and $\text{rot}_Q(x)$ will be used to refer to the 3D position vector and the rotation quaternion of a 3D pose or isometry $x$.
When referring to the rotation as a 3D rotation matrix, the function $\text{rot}_{RM}(x)$ will be used instead.
When using $x$ in calculations, it will refer to the pose's 3D isometry containing both the position vector as translation as well as the rotation quaternion.
The binary operator $\circ$ will be used to concatenate these isometries.
The inverse of an isometry will be denoted as $x^{-1}$.

Some functions will be used to calculate gradients of a 3D isometry $x$, such as $\frac{\text{d}q}{\text{d}R}(x)$, which is computed as follows using the trace function $\text{tr}(M)$ of 3x3 matrices:

\begin{align}
	s = \frac{1}{2}\sqrt{\text{tr}(\text{rot}_{RM}(x)) + 1}
\end{align}

\begin{align}
	a_1 = -\frac{\text{rot}_{RM}(x)_{21}-\text{rot}_{RM}(x)_{12}}{32s^3}
\end{align}

\begin{align}
	a_2 = -\frac{\text{rot}_{RM}(x)_{02}-\text{rot}_{RM}(x)_{20}}{32s^3}
\end{align}

\begin{align}
	a_3 = -\frac{\text{rot}_{RM}(x)_{10}-\text{rot}_{RM}(x)_{01}}{32s^3}
\end{align}

\begin{align}
	b = \frac{1}{4s}
\end{align}

\begin{align}
	\frac{\text{d}q}{\text{d}R}(x) =
	\begin{pmatrix}
		a_1 & 0 &  0 &  0 & a_1 & b & 0 & -b & a_1\\
		a_2 & 0 & -b &  0 & a_2 & 0 & b &  0 & a_2\\
		a_3 & b &  0 & -b & a_3 & 0 & 0 &  0 & a_3
	\end{pmatrix}
\end{align}

It will always be assumed that the condition $\text{tr}(\text{rot}_{RM}(x)) > 0$ holds, i.e. we are dealing with proper rotations.

Another function used in this context is $\text{skew}(x)$, computed as follows using a 3D vector or position $x$:

\begin{align}
	\text{skew}(x) =
	\begin{pmatrix}
		   0 &  x_z & -x_y\\
		-x_z &    0 &  x_x\\
		 x_y & -x_x &    0
	\end{pmatrix},
\end{align}

where $x_x$, $x_y$ and $x_z$ are the first, second and third vector components, respectively. The $\text{skew}(x)$ can be interpreted as the matrix 
representation of the cross product with $x$.

### Position3D

The `Position3D` factor involves one `VehicleVariable` $x_v$.
Given the measurement $x_m$, the Jacobian matrix $J$ is calculated as follows:

\begin{align}
	E = x_m^{-1} \circ x_v
\end{align}

\begin{align}
	\frac{\text{d}te}{\text{d}qj} = \frac{\text{d}q}{\text{d}R}(E) \circ
	\begin{pmatrix}
		\text{skew}(\text{rot}_{RM}(E)_0)\\
		\text{skew}(\text{rot}_{RM}(E)_1)\\
		\text{skew}(\text{rot}_{RM}(E)_2)
	\end{pmatrix}
\end{align}

\begin{align}
	J =
	\begin{pmatrix}
		\boldsymbol{E} &               \boldsymbol{0}\\
		\boldsymbol{0} & \boldsymbol{\frac{\text{d}te}{\text{d}qj}}
	\end{pmatrix},
\end{align}

where $M_i$ refers to the column at index $i$ of the matrix $M$.

The error vector $e$ is computed as follows:

\begin{align}
	e = x_m^{-1} \circ x_v
\end{align}

As mentioned before, the scalar part of the quaternion is removed in $e$. $e$ is then interpreted as a six-dimensional
vector with the top three entries for its position and the bottom three entries for its rotation.

### Odometry3D
The `Odometry3D` factor involves two `VehicleVariable`s $x_i$ and $x_j$.
Given the measurement $x_{ij}$, the Jacobian matrix $J$ is calculated as follows:

\begin{align}
	A = x_{ij}^{-1}
\end{align}

\begin{align}
	B = x_i^{-1} \circ x_j
\end{align}

\begin{align}
	E = A \circ B
\end{align}

\begin{align}
	\frac{dte}{dqi} = \text{rot}_{RM}(A) \circ \text{skew}(\text{pos}(B))
\end{align}

\begin{align}
	\frac{dre}{dqi} = \frac{dq}{dR}(E) \circ
	\begin{pmatrix}
		\boldsymbol{\text{rot}_{RM}(A) \circ \text{skew}(\text{rot}_{RM}(B)_0)^T}\\
		\boldsymbol{\text{rot}_{RM}(A) \circ \text{skew}(\text{rot}_{RM}(B)_1)^T}\\
		\boldsymbol{\text{rot}_{RM}(A) \circ \text{skew}(\text{rot}_{RM}(B)_2)^T}
	\end{pmatrix}
\end{align}

\begin{align}
	J_i =
	\begin{pmatrix}
	\boldsymbol{-\text{rot}_{RM}(A)} & \boldsymbol{\frac{dte}{dqi}}\\
	\boldsymbol{0} & \boldsymbol{\frac{dre}{dqi}}
	\end{pmatrix}
\end{align}

\begin{align}
	\frac{dre}{dqj} = \frac{dq}{dR}(E) *
	\begin{pmatrix}
		\boldsymbol{\text{skew}(\text{rot}_{RM}(E)_0)}\\
		\boldsymbol{\text{skew}(\text{rot}_{RM}(E)_1)}\\
		\boldsymbol{\text{skew}(\text{rot}_{RM}(E)_2)}
	\end{pmatrix}
\end{align}

\begin{align}
	J_j =
	\begin{pmatrix}
		\boldsymbol{\text{rot}_{RM}(E)} & \boldsymbol{0}\\
		          \boldsymbol{0} & \boldsymbol{\frac{dre}{dqj}}
	\end{pmatrix}
\end{align}

\begin{align}
	J =
	\begin{pmatrix}
		\boldsymbol{J_i} & \boldsymbol{J_j}
	\end{pmatrix},
\end{align}

where $M_i$ refers to the column at index $i$ of the matrix $M$.

The error vector $e$ is computed as follows:

\begin{align}
	e = x_{ij}^{-1} \circ x_i^{-1} \circ x_j
\end{align}

As mentioned in section ?, the scalar part of the quaternion is removed in $e$. $e$ is then interpreted as a six-dimensional vector with the
top three entries for its position and the bottom three entries for its rotation.

### Observation3D
The `Observation3D` factor involves one `VehicleVariable` $x_i$ and one `LandmarkVariable` $x_j$.
In the following, the binary operator $\circ$ denotes the transformation of the position given by the right operand by the isometry given by the left operand.

With the measurement $x_{ij}$, the Jacobian matrix $J$ is calculated as follows:

\begin{align}
	J =
	\begin{pmatrix}
		\boldsymbol{-I_3} & \boldsymbol{\text{skew}(\text{pos}(x_i^{-1} \circ \text{pos}(x_j)))^T} & \boldsymbol{\text{rot}_{RM}(x_i^{-1})}
	\end{pmatrix}
\end{align}

Finally, the error vector $e$ is computed as follows:

\begin{align}
	e = \text{pos}(x_i^{-1} \circ \text{pos}(x_j)) - \text{pos}(x_{ij})
\end{align}



